In [1]:
import sys
from pathlib import Path

import torch
import torch.nn.functional as F
import numpy as np
from tqdm.notebook import tqdm
from sklearn.metrics import accuracy_score
from torch.utils.data import DataLoader

# Import from your codebase
from CLAPWrapper import CLAPWrapper
from datasets.esc50 import ESC50

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")

Device: cpu


In [2]:
print("\n" + "="*80)
print("Loading ESC50 Dataset")
print("="*80)

root_path = "./data"
dataset = ESC50(root=root_path, download=True)
print(f"✅ Dataset loaded: {len(dataset)} samples")
print(f"   Classes: {len(dataset.classes)} categories")
print(f"   Sample classes: {dataset.classes[:5]}")

# Prepare text prompts
prompt = 'this is the sound of '
text_labels = [prompt + x for x in dataset.classes]
print(f"\n📝 Text prompts: {len(text_labels)} classes")


Loading ESC50 Dataset
Loading audio files


2000it [00:00, 17482.71it/s]

✅ Dataset loaded: 2000 samples
   Classes: 50 categories
   Sample classes: ['airplane', 'breathing', 'brushing teeth', 'can opening', 'car horn']

📝 Text prompts: 50 classes


In [3]:
# ============================================================================
# STEP 3: Initialize Models
# ============================================================================
print("\n" + "="*80)
print("Initializing Models")
print("="*80)

# Residual config con pc_weights = 1.0 (identità)
residual_config = {
    'n_components_ratio': .01,
    'reweight_factor': 3.0,
    'target_layers': [3],  # Layers dove applicare reweighting
    'analysis_mode': True
}

print("\n🔧 Loading CLAP Standard...")
clap_standard = CLAPWrapper(
    version='2023',  # or '2022'
    use_cuda=torch.cuda.is_available(),
    type='classic'
)

print("\n🔧 Loading ResiDualCLAP...")
clap_residual = CLAPWrapper(
    version='2023',
    use_cuda=torch.cuda.is_available(),
    type='residual',
    residual_config=residual_config
)


Initializing Models

🔧 Loading CLAP Standard...

🔧 Loading ResiDualCLAP...
[2, 2, 6, 2]
🔍 Detecting layer dimensions...
  ✓ 0: torch.Size([1, 1024, 192])
  ✓ 1: torch.Size([1, 256, 384])
  ✓ 2: torch.Size([1, 64, 768])
  ✓ 3: torch.Size([1, 64, 768])
  ✓ layer_3: 768D → 7 PCs


In [4]:
print("\n" + "="*80)
print("Fitting PCA Components")
print("="*80)

# Prepare audio samples for PCA fitting
print("Collecting samples for PCA fitting (max 200 samples)...")

# Create a simple dataloader wrapper per PCA fitting
class SimpleAudioDataset:
    def __init__(self, wrapper, esc50_dataset, max_samples=1000):
        self.wrapper = wrapper
        self.audio_paths = []
        for i in range(min(max_samples, len(esc50_dataset))):
            audio_path, _, _ = esc50_dataset[i]
            self.audio_paths.append(audio_path)
    
    def __len__(self):
        return len(self.audio_paths)
    
    def __getitem__(self, idx):
        audio_tensor = self.wrapper.load_audio_into_tensor(
            self.audio_paths[idx],
            self.wrapper.args.duration,
            resample=True
        )
        # ✅ Assicurati sia 1D
        if audio_tensor.dim() > 1:
            audio_tensor = audio_tensor.squeeze()
        
        return audio_tensor

# Create dataset and loader
pca_dataset = SimpleAudioDataset(clap_residual, dataset, max_samples=200)
pca_loader = DataLoader(
    pca_dataset, 
    batch_size=16, 
    shuffle=False,
    num_workers=0,  # Start with 0 for debugging
    pin_memory=False
)

# Fit PCA
print(f"Fitting PCA on {len(pca_dataset)} samples...")
variance_ratios = clap_residual.clap.audio_encoder.base.htsat.fit_spectral_layers(
    pca_loader,
    max_samples=100
)

print("\n📊 PCA Variance Ratios:")
for layer_name, ratios in variance_ratios.items():
    print(f"   {layer_name}: Top 5 components = {ratios[:5]}")


Fitting PCA Components
Fitting PCA on 200 samples...

🔍 PHASE 1: Collecting Hidden States from HTSAT Layers
Target layers: ['layer_3']
Max samples to collect: 100
Batches in dataloader: 13



✓ Collection completed in 21.70s
  • Total samples processed: 112
  • Successful batches: 7
  • Failed batches: 0
  • Samples per second: 5.2

📦 Collected data per layer:
  • layer_3: 7 batches, 7,168 tokens

📊 PHASE 2: Fitting PCA Components


Fitting PCA:   0%|          | 0/1 [00:00<?, ?layer/s]


🔧 Processing layer_3:
  • Concatenating 7 batches...
  • Combined shape: torch.Size([7168, 768]) (7,168 samples)
  • Memory usage: 21.0 MB
  • Running PCA decomposition...


Fitting PCA: 100%|██████████| 1/1 [00:00<00:00,  1.92layer/s]

  • PCA completed in 0.51s

  📈 Variance Analysis:
     • Total components: 7
     • Top 5 variances: ['0.0721', '0.0603', '0.0539', '0.0486', '0.0451']
     • Cumulative variance:
        - 50% variance: 8/7 components (114.3%)
        - 70% variance: 8/7 components (114.3%)
        - 80% variance: 8/7 components (114.3%)
        - 90% variance: 8/7 components (114.3%)
        - 95% variance: 8/7 components (114.3%)

     Top 10 components bar:
        PC 1: ████████████████████████████████████████ 0.0721
        PC 2: █████████████████████████████████ 0.0603
        PC 3: █████████████████████████████ 0.0539
        PC 4: ██████████████████████████ 0.0486
        PC 5: █████████████████████████ 0.0451
        PC 6: ██████████████████████ 0.0399
        PC 7: ███████████████████ 0.0344

✓ PCA Fitting Completed Successfully
Total time: 22.23s

Spectral layers ready for reweighting:
  • layer_3: ✓ FITTED

📊 PCA Variance Ratios:
   layer_3: Top 5 components = [0.07210616 0.06026053 0.053

In [5]:
# Get text embeddings ONCE for all classes
text_embeddings = clap_standard.get_text_embeddings(text_labels)
print(f"   Text embeddings shape: {text_embeddings.shape}")

# Test on subset first (use full dataset later)
test_size = 100  # Start with 200 samples for speed
print(f"\n📊 Testing on {test_size} samples...")

y_preds_baseline, y_labels = [], []

for i in tqdm(range(test_size), desc="Baseline"):
    # Get audio file path and label
    audio_path, target, one_hot_target = dataset[-(i+1)]
    
    # Get audio embedding
    audio_embedding = clap_standard.get_audio_embeddings([audio_path], resample=True)
    
    # Compute similarity
    similarity = clap_standard.compute_similarity(audio_embedding, text_embeddings)
    
    # Get prediction
    y_pred = F.softmax(similarity.detach().cpu(), dim=1).numpy()
    y_preds_baseline.append(y_pred)
    y_labels.append(one_hot_target.detach().cpu().numpy())

# Calculate accuracy
y_labels_array = np.concatenate(y_labels, axis=0)
y_preds_baseline_array = np.concatenate(y_preds_baseline, axis=0)

baseline_acc = accuracy_score(
    np.argmax(y_labels_array, axis=1), 
    np.argmax(y_preds_baseline_array, axis=1)
)

print(f"\n✅ Baseline Accuracy: {baseline_acc:.3f} ({baseline_acc*100:.1f}%)")

   Text embeddings shape: torch.Size([50, 1024])

📊 Testing on 100 samples...


Baseline:   0%|          | 0/100 [00:00<?, ?it/s]


✅ Baseline Accuracy: 0.920 (92.0%)


In [5]:
# Get text embeddings ONCE for all classes
text_embeddings = clap_residual.get_text_embeddings(text_labels)
print(f"   Text embeddings shape: {text_embeddings.shape}")

# Test on subset first (use full dataset later)
test_size = 100  # Start with 200 samples for speed
print(f"\n📊 Testing on {test_size} samples...")

y_preds_baseline, y_labels = [], []

for i in tqdm(range(test_size), desc="Baseline"):
    # Get audio file path and label
    audio_path, target, one_hot_target = dataset[-(i+1)]
    
    # Get audio embedding
    audio_embedding = clap_residual.get_audio_embeddings([audio_path], resample=True)
    
    # Compute similarity
    similarity = clap_residual.compute_similarity(audio_embedding, text_embeddings)
    
    # Get prediction
    y_pred = F.softmax(similarity.detach().cpu(), dim=1).numpy()
    y_preds_baseline.append(y_pred)
    y_labels.append(one_hot_target.detach().cpu().numpy())

# Calculate accuracy
y_labels_array = np.concatenate(y_labels, axis=0)
y_preds_baseline_array = np.concatenate(y_preds_baseline, axis=0)

baseline_acc = accuracy_score(
    np.argmax(y_labels_array, axis=1), 
    np.argmax(y_preds_baseline_array, axis=1)
)

print(f"\n✅ Baseline Accuracy: {baseline_acc:.3f} ({baseline_acc*100:.1f}%)")

   Text embeddings shape: torch.Size([50, 1024])

📊 Testing on 100 samples...


Baseline:   0%|          | 0/100 [00:00<?, ?it/s]


✅ Baseline Accuracy: 0.910 (91.0%)


In [5]:
audio_path, target, one_hot_target = dataset[0]

In [6]:
clap_residual.get_audio_embeddings([audio_path], resample=True)

VAIII, reweiting layer 0
VAIII, reweiting layer 1
VAIII, reweiting layer 2
VAIII, reweiting layer 3


tensor([[ 0.8734,  0.3351, -0.7393,  ...,  2.1447,  1.0662, -0.1054]])

In [7]:
clap_standard.get_audio_embeddings([audio_path], resample=True)

tensor([[ 0.7623,  0.2343, -0.5101,  ...,  1.8940,  0.9414, -0.0119]])